Загрузить данные из таблицы test.xlsx.  
Проанализировать текст в колонке text, используя метод обработки текста.  
Сгенерировать вероятный краткий ответ из всего текста (состоящий из нескольких слов) и записать его в колонку answer.  

Пример:  
Текст1 - Удовлетворить  
Текст2 - Производство окончено  
Текст3 - Отказ, ИП приостановлено  
Текст4 - Отказ, невозможно установить местонахождение должника  

Такую задачу можно просто сделать через LLM, через OpenRouter с бесплатными моделями, но тут будет хромать скорость и лимиты запросов, доступность сервиса. Если же сделать через платный API, то нужны постоянные дополнительные затраты.
Так что можно сделать через более простые способы, попробовать линейные модели и бустинг модели, базовые обработки текста.

### Параметры

In [1]:
import pandas as pd
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

os.chdir('../../')

In [2]:
df = pd.read_csv('data/processed/rule_classified.csv')

Будем применять линейнеые и бустинг модели: Naive Bayes, Logistic Regression, SVM, Decision Tree, RandomForest, GradientBoosting, HistGradientBoostingClassifier, CatBoost, XGBoost, LightGBM.  
Может ещё в конце языковые модели.

In [3]:
df_labeled = df[df['answer'].notna()].copy()
# original_indices_labeled = df_labeled.index.copy() # Для последующего объеденения с предсказаниями
df_labeled = df_labeled.reset_index(drop=True)

df_labeled['answer'].value_counts()

answer
Отказ                                     217
Частично удовлетворено                    138
Обращение рассмотрено                     130
Взыскание обращено                        106
Запрос направлен                           80
Возбуждено исполнительное производство     76
Постановление вынесено                     37
Удовлетворено                              27
Заявления и жалобы рассматриваются         26
Объявлен исполнительный розыск             20
Применены меры для исполнения              19
Запрет действий                            14
Name: count, dtype: int64

In [4]:
from src.classification_utils import split_data, encode_labels

X_train, X_test, y_train, y_test = split_data(df_labeled)
y_train_enc, y_test_enc, le = encode_labels(y_train, y_test)

print(f"{'='*15}Размеры классов тренировочной выборки:{'='*15}\n{y_train.value_counts()}\n")
print(f"{'='*15}Размеры классов тестовой выборки:{'='*15}\n{y_test.value_counts()}")

===============Размеры классов тренировочной выборки:===============
answer
Отказ                                     152
Частично удовлетворено                     97
Обращение рассмотрено                      91
Взыскание обращено                         74
Запрос направлен                           56
Возбуждено исполнительное производство     53
Постановление вынесено                     26
Удовлетворено                              19
Заявления и жалобы рассматриваются         18
Объявлен исполнительный розыск             14
Применены меры для исполнения              13
Запрет действий                            10
Name: count, dtype: int64

===============Размеры классов тестовой выборки:===============
answer
Отказ                                     65
Частично удовлетворено                    41
Обращение рассмотрено                     39
Взыскание обращено                        32
Запрос направлен                          24
Возбуждено исполнительное производство    23
Пост

In [5]:
import pickle
from src.classifiers import get_classifiers

# Получаем список векторизированных данных
with open('vectors/vectorizers.pkl', 'rb') as f:
    vectorizers = pickle.load(f)

# Получаем список моделей и параметры их обучения
classifiers = get_classifiers(le)

### Baseline

Попробуем линейные модели, если они будут плохи, откинем их.  
Скорее всего бустинг модели будут хороши в этой задаче, а может и слишком сложны.  
Разница между ними под капотом большая, но можно попробовать все, сравнить результаты.

Обучим алгоритмы на тех данных, которые уже имеют верные метки

Может стоит попробывать соединить tf-idf с word2vec, потому-что часто попадаются по всему датасету шаблонные фразы, а word2vec может уловить семантическую близость между разными формулировками

#### Без MLFlow, старый

In [6]:
%%script cmd /c ""
# Оставил вариант без mlflow, который я делал как первый тест моделей
from sklearn.metrics import classification_report
import numpy as np
import gc

# При обучении выдаёт разные warning, можно игнорировать, так как не мешают. Может быть из-за попытки сделать часть моделей на гпу
# - UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Исправляется через SMOTE из-за редкости классов
# - DataConversionWarning: A column-vector y was passed when a 1d array was expected. Точно подаётся одномерный массив, но всё равно предупреждение
# - UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names. Подаётся np.array, но всё равно предупреждение

for vec_name, (train_path, test_path) in vectorizers.items():
    print(f"\n{'|'*50}{vec_name}{'|'*50}\n")

    # Загружаем только для текущей итерации, памяти ради
    if train_path.endswith('.npz'):
        from scipy.sparse import load_npz
        X_train_vec = load_npz(train_path)
        X_test_vec = load_npz(test_path)
    else: # .npy
        X_train_vec = np.load(train_path)
        X_test_vec = np.load(test_path)

    for name, classifier in classifiers:
        print(f"\n{'='*90}\nМодель: {name} \n{'='*90}\n") 
        
        classifier.fit(X_train_vec, y_train_enc)
        y_pred = classifier.predict(X_test_vec)
        y_pred_labels = le.inverse_transform(y_pred)
        print(classification_report(y_test, y_pred_labels, digits=3))

    # Освобождаем память
    del X_train_vec, X_test_vec
    gc.collect()

`UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use zero_division parameter to control this behavior. _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])`

Ошибка возникает, так как модели не предсказывают ни одного примера редких классов, более сложные модели же попытались, хоть и не очень удачно по precision, recall, f1, но немного лучше чем просто угадывание по всем классам

#### С MLFlow

Для запуска MLFlow ввести в cmd:
```python
mlflow server `
  --backend-store-uri sqlite:///mlruns/mlflow.db `
  --default-artifact-root ./mlruns/artifacts `
  --host 127.0.0.1 `
  --port 5000
```

In [7]:
from src.classification_utils import load_np_arrays, crossval_report, log_report
import mlflow
import warnings
from tqdm.notebook import tqdm
import os

os.environ['MLFLOW_SUPPRESS_PRINTING_URL_TO_STDOUT'] = '1' # так как выводит view run и view experiment из-за того, что запустил отдельный сервер
warnings.filterwarnings('ignore')

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('text_classification')

pbar_vectorizers = tqdm(total=len(vectorizers), desc='Векторизаторы', position=0)
pbar_classifiers = tqdm(total=len(classifiers), desc='Классификаторы', position=1)

for vec_name, (train_path, test_path) in vectorizers.items(): # items так как словарь
    X_train_vec, X_test_vec = load_np_arrays(train_path, test_path)

    pbar_vectorizers.set_description(f'Векторизатор: {vec_name}')
    pbar_classifiers.reset()
    
    for classifier_name, classifier in classifiers:
        pipeline = classifier
        report = crossval_report(classifier, X_train_vec, y_train, y_train_enc, le, pipeline=pipeline)
        log_report(report, vec_name, classifier_name, resampler_name='no_resampler')

        pbar_classifiers.set_description(f'Классификатор: {classifier_name}')
        pbar_classifiers.update(1)

    pbar_vectorizers.update(1)

Векторизаторы:   0%|          | 0/6 [00:00<?, ?it/s]

Классификаторы:   0%|          | 0/8 [00:00<?, ?it/s]

Новые метрики уже больше похожи на правду, почти нету метрик с идеальным показателем 1.0.  
Также линейные модели лучше предсказывают самый редкий класс, чем бустинговые модели, кроме Gradient Boosting. Это может быть, потому-что бустинг модели пытаются уменьшить ошибку на всех классах, пренебрегая самым редким классом спустя множество бустинг моделей.

- Можно попробовать увеличить размер минорного (миноритарного) класса через его дублирование (RandomOverSampler)(более простой) или через SMOTE.  
- Также можно попробовать изменить вес класса вручную для каждой модели, чем и можно заняться перед SMOTE.  
- Focal Loss для поддерживаемых моделей.
- Сравнить TF-IDF с Word2Vec, GloVe, FastText.
- Попровать сделать что-то с ruBERT.

Думал аугментировать данные, но замена синонимами, перефразированием в таком датасете может не представлять сам датасет, так как фразы шаблонны.  
Можно сделать синтетические примеры через LLM, но это напоследок, если всё будет плохо с малыми классами.

### Классификация неразмеченного текста

#### Через ансамбль

In [ ]:
df_unlabeled = df[df['answer'].isna()].copy()
# original_indices_unlabeled = df_labeled.index.copy()
df_unlabeled = df_unlabeled.reset_index(drop=True) # Нужно обнулить индексы, так как предсказывает только 100 из 322 почему-то
X_unlabeled = df_unlabeled['lemmatized_text']

predictions_all = {}
for name, pipeline in trained_pipelines.items():
    preds = pipeline.predict(X_unlabeled)
    predictions_all[name] = preds.ravel() if hasattr(preds, "ravel") else preds # ravel() так как CatBoost выдаёт двумерный массив, вместо одномерного

predictions_df = pd.DataFrame(predictions_all)
# # Перевернуть столбцы, чтобы из-за mode, которая при всех разных вариантов выбирает первый столбце в списке, выбиралась лучшая модель - SVM
# predictions_df = predictions_df[['SVM', 'Logistic Regression', 'Naive Bayes']]

# Преобразуем обратно в текстовые метки
for col in predictions_df.columns:
    predictions_df[col] = le.inverse_transform(predictions_df[col])

# Выбрать наиболее частый ответ среди всех строк
df_unlabeled['answer'] = predictions_df.mode(axis=1)[0]


print("Датафрейм с предсказаниями от всех моделей:")
print(predictions_df.head())

print("\nКоличество предсказанных классов:", df_unlabeled['answer'].value_counts())

In [ ]:
df_unlabeled[['answer', 'lemmatized_text']].head()

То есть любые неизвестные текста модели относят к "Обращение рассмотрено", что говорит о том, что классов недостаточно и нужно придумывать что-то иное, но, по-крайне мере модели предсказывают классы с высокими метриками

#### Итоговый датасет

In [ ]:
df_labeled.index = df[df['answer'].notna()].index
df_unlabeled.index = df[df['answer'].isna()].index

In [ ]:
df_result = pd.concat([df_labeled, df_unlabeled]).sort_index()

print(df.index)
print(df_result.index)

Индексы ~совпадают

In [ ]:
# Сохраню в xlsx, так как много строк имеют \n и \r символов, что ломает разделение строк в csv
df_result[['Id', 'text', 'answer']].to_excel('data/processed/result.xlsx', index=False)
df_result.to_excel('data/processed/result_detailed.xlsx', index=False)